# Siamese Network for Outfit Compatibility


To assess how well different clothing items pair together, we implemented a **Siamese Network** that learns to model outfit compatibility using pairwise embeddings.

This model takes input from our previously trained ResNet-18 classifier (Model 1), which generates 512-dimensional feature embeddings for each clothing item.

### Inputs:
- `feature_model.pth`: Pre-trained feature extractor (ResNet-based)
- `training_pairs.json`: JSON file containing labeled pairs of compatible and incompatible outfits

These inputs are used to train the Siamese Network to compare two items and learn a 128-dimensional compatibility embedding that captures how visually and stylistically well two items go together.

### Feature Extraction and Training:
- For each clothing item pair, 512-D features are extracted using Model 1
- These features are passed through a Siamese architecture to learn pairwise relationships
- The output embeddings are trained using contrastive loss, encouraging similar pairs to be close in embedding space and dissimilar pairs to be far apart

### Compatibility Scoring:
Once trained, the Siamese model evaluates any pair of items by computing the Euclidean distance between their compatibility embeddings:
- Smaller distances indicate higher compatibility
- This allows the system to rank or filter clothing items during outfit generation

### Output:
- The final trained model is saved as `siamese_model.pth`
- This model is used downstream to support color and style pairing logic within the Outfit Pair Builder module

### Evaluation

To evaluate the effectiveness of our Siamese Network in modeling outfit compatibility, we conducted an embedding-based similarity search using the final trained model.

We began by loading precomputed 512-dimensional feature embeddings generated from Model 1, and then passed them through the trained Siamese model to obtain 128-dimensional compatibility embeddings. These embeddings were stored and indexed using FAISS for efficient similarity lookup.

We then used a query-based evaluation strategy, where for a given clothing item , we retrieved the top-k most compatible items from other categories . Compatibility was determined by computing the Euclidean distance between embeddings — the smaller the distance, the more compatible the items were assumed to be.

To validate this, we used real wardrobe metadata and checked if the top matches shared consistent style tags . This ensured that retrieval wasn't just based on visual similarity but also reflected meaningful stylistic compatibility.

### Visualisation 

#### 1. Distance Histogram of Siamese Embeddings

<img src="images/histogram.png" alt="Distance Histogram of Positive and Negative Pairs" width="600">
This histogram visualizes the distribution of Euclidean distances between clothing item pairs in the learned Siamese embedding space. The pairs are split into:

As shown in the image, the red distribution (negative pairs) peaks on the left, around 0.2–0.4
The green distribution (positive pairs) peaks around 0.7–0.8
Despite some overlap in the 0.5–0.6 range, the histogram shows a clear separation between compatible and incompatible items, demonstrating that the model is successfully learning to distinguish stylistically matched items from mismatched ones.

This evaluation confirms the contrastive loss objective was effective in shaping the embedding space for meaningful outfit compatibility scoring.

#### 2. t-SNE Visualization of Embedding Space

<img src="images/tsne.png" alt="t-SNE Visualization of Embedding Space" width="600">

The clear clustering of items by type indicates that the Siamese model has learned a meaningful embedding structure — grouping visually or stylistically similar items close together, while separating distinct types.

For instance:

T-shirts (orange) form a well-defined cluster in the bottom-left, clearly separated from other categories.
Shirts (red) are mostly grouped near each other in the top-middle.
Bottomwear items like pants (blue) and shorts (green) overlap slightly, but still form recognizable regions.
This shows that the model is successfully learning category-aware compatibility features, which improves its ability to suggest valid top-bottom combinations during outfit pairing.

#### Top query match 
This is how the top outfit matches look like when provided the query 
<img src="images/model2_eval.png" alt="t-SNE Visualization of Embedding Space" width="600">